In [ ]:
# Title: Fig3----
# A régler: Si on prend 1978 pas tous les fichiers dans une année donc pb
# Normaliser? donnes entre 1980 et 210 
# Arthur BARREAU: 06 Février 2025

In [2]:
system("conda install -y r-sf=0.9_8")

In [8]:
system("conda install -y r-terra=1.2_10")

In [9]:
system("conda install -y r-dplyr")

In [10]:
system("conda install -y r-stringr")

In [11]:
system("conda install -y r-RColorBrewer")

In [ ]:
# Title: Fig3----
# A régler: Si on prend 1978 pas tous les fichiers dans une année donc pb
# Normaliser? 
# Entrée doit venir de Galaxy 
# Arthur BARREAU: 06 Février 2025

library(terra)
library(dplyr)
library(stringr)
library(RColorBrewer)
library(sf)
library(jsonlite) 

# 2 VARIABLE ----
months_days <- list(
  Jan = 31, Feb = 28, Mar = 31, Apr = 30, May = 31, Jun = 30, 
  Jul = 31, Aug = 31, Sep = 30, Oct = 31, Nov = 30, Dec = 31
)

# 3 FUNCTION ----

"""
Create directories for storing TIFF and Shapefile data.

Args:
    year (int): The year for which directories will be created.
    month (str): The month for which directories will be created.

Returns:
    list: A list containing paths for TIFF, ZIP, Shapefile, and mean TIFF directories.
"""
create_directories <- function(year, month) {
  tif_path <- sprintf("%s_%s_tif_folder", year,month)
  zip_path <- sprintf("%s_%s_zip_folder", year,month)
  shp_path <- sprintf("%s_%s_shapefile_folder", year,month)
  tif_mean_path <- sprintf("%s_%s_folder_tif_1981_2010", year,month)
  dir.create(tif_path, showWarnings = FALSE, recursive = TRUE)
  dir.create(zip_path, showWarnings = FALSE, recursive = TRUE)
  dir.create(shp_path, showWarnings = FALSE, recursive = TRUE)
  dir.create(tif_mean_path, showWarnings = FALSE, recursive = TRUE)
  return(list(tif_path = tif_path, zip_path = zip_path, shp_path = shp_path, tif_mean_path=tif_mean_path))
}

"""
Download daily TIFF files for a given month and year.

Args:
    path (str): The directory where TIFF files will be stored.
    year (int)
    month (str)
    month_index (int): The numerical index of the month.

"""
download_tiff_files <- function(path, year, month, month_index) {
  month_index <- match(month, names(months_days))  
  month_folder <- sprintf("%02d_%s", month_index, month) 
  
  for (day in 1:months_days[[month]]) {
    date_str <- sprintf("%s%02d%02d", year, month_index, day)  
    url <- paste0("https://noaadata.apps.nsidc.org/NOAA/G02135/south/daily/geotiff/", year, "/", month_folder, "/S_", date_str, "_concentration_v3.0.tif")
    destfile <- file.path(path, sprintf("S_%s_concentration.tif", date_str))
    tryCatch({
    download.file(url, destfile)
    },error=function(e){})
  }
}


"""
Compute the monthly mean raster from a set of TIFF files.

Args:
    path (str): The directory containing the TIFF files.
    year (int): The year to consider for computing the mean.
    month_index (int)

Returns:
    rast: A raster object representing the monthly mean.
"""
compute_monthly_mean <- function(path, year, month_index) {
  tif_files <- list.files(path, pattern = "\\.tif$", full.names = TRUE)
  rasters <- rast(tif_files)
  rasters[rasters == 2550] <- NA  # Remplacer les valeurs 2550 par NA
  mean_raster <- mean(rasters, na.rm = TRUE)  
  return(mean_raster)
}


"""
Download shapefiles for a given month.

Args:
    zip_path (str): The directory where the zip file will be saved.
    shp_path (str): The directory where shapefiles will be extracted.
    month_index (int)
"""
download_shapefiles <- function(zip_path, shp_path, month_index) {
  month_index <- sprintf("%02d", month_index) 
  url <- paste0("https://noaadata.apps.nsidc.org/NOAA/G02135/south/monthly/shapefiles/shp_median/median_extent_S_",
                month_index, "_1981-2010_polyline_v3.0.zip")
  destfile <- file.path(zip_path, sprintf("median_extent_S_%s.zip", month_index))
  tryCatch({
    download.file(url, destfile)
    unzip(destfile, exdir = shp_path) 
    },error=function(e){})
}

"""
Merge shapefiles and compute mean polygon from the shapefiles.

Args:
    shp_path (str): The directory where shapefiles are stored.

Returns:
    list: A list of polygons read from the shapefiles.
"""
process_shapefiles <- function(shp_path) {
  shp_files <- list.files(shp_path, pattern = "\\.shp$", full.names = TRUE)
  polygons <- lapply(shp_files, st_read)
  return(polygons)
}

"""
Download daily TIFF files for the reference period 1981-2010.

Args:
    path (str): The directory where TIFF files will be stored.
    start_year (int): The starting year of the reference period.
    end_year (int): The ending year of the reference period.
    month_index (int)

"""
download_tiff_1981_2010 <- function(path, start_year, end_year, month_index) {
  month_index <- sprintf("%02d", month_index) 
  base_url <- "https://noaadata.apps.nsidc.org/NOAA/G02135/south/monthly/geotiff/"
  
  months <- c("01_Jan", "02_Feb", "03_Mar", "04_Apr", "05_May", "06_Jun",
              "07_Jul", "08_Aug", "09_Sep", "10_Oct", "11_Nov", "12_Dec")
  month_folder <- months[as.integer(month_index)]
  
  for (year in start_year:end_year) {
    date_str <- sprintf("S_%s%s_concentration_v3.0.tif", year, month_index)
    url <- paste0(base_url, month_folder, "/", date_str)
    destfile <- file.path(path, date_str)
    tryCatch({
    download.file(url, destfile, mode = "wb")
    },error=function(e){})
  }
}


"""
Compute the mean raster for the reference period 1981-2010.

Args:
    path (str): The directory containing the TIFF files.
    month_index (int): The month index for which the mean will be computed.

Returns:
    rast: A raster object representing the mean for the reference period.
"""
compute_mean_raster_1981_2010 <- function(path,month_index) { 
  month_index <- sprintf("%02d", month_index) 
  files <- list.files(path, pattern = "\\.tif$", full.names = TRUE)
  rasters <- rast(files)
  mean_raster_1981_2010 <- mean(rasters, na.rm = TRUE)  
  return(mean_raster_1981_2010)
}

"""
Create color palettes and breaks for raster visualization.

Returns:
    list: A list containing the color palette and breaks for visualization.
"""
create_colors_and_breaks <- function() {
  breaks <- c(0,1, seq(150, 1000, by = 50),1050, 2531, 2541)

  colors_glacier <- rev(c("white",
    "#F7FCFF", "#E4F4FE", "#D0ECFE", "#BCE4FE", "#A8DCFD",
    "#94D5FD", "#7DCCFD", "#6AC4FC", "#57BCFC", "#45B4FC",
    "#31ABFC", "#23A3FC", "#1A9BFC", "#1994F9", "#178CF2",
    "#1684EB", "#137AE3", "#093C70" ))

  colors <- c(
    rgb(9, 60, 112, maxColorValue = 255),
    colors_glacier,  # Dégradé bleu glacier (1 à 1000)
    rgb(0, 0, 0, maxColorValue = 255),      # Ligne côtière noire (2530)
    rgb(119, 119, 119, maxColorValue = 255) # Terre grise (2540)
  )
  return(list(colors = colors, breaks = breaks))
}

"""
Visualize the raster and overlay shapefile polygons.

Args:
    raster (rast): The raster data to be visualized.
    colors (list): The color palette for the raster.
    breaks (list): The breaks for the raster color scale.
    polygons (list): A list of polygons to overlay on the raster.
    year (int)
    month (str)

"""
save_plot <- function(raster, colors, breaks, polygons, year, month) {
  plot(raster, col = colors, breaks = breaks, legend = FALSE, axes = FALSE, box = FALSE, mar = c(0, 0, 0, 0))
  title(main = sprintf("%s_%s", month,year),line=-5,cex.main = 2,col.main="white")
  plot(st_geometry(polygons[[1]]), add = TRUE, pch = 21, col = 'red', cex = 1.5)
}

# ANOMALY ---------------

"""
Filter raster values to set specific ranges to zero.

Args:
    raster (rast): The raster to be filtered.

Returns:
    rast: The filtered raster.
"""
filter_raster_values <- function(raster) {
  raster[] <- ifelse(raster[] >= 1 & raster[] <= 150, 0, raster[])
  return(raster)
}

"""
Compute the anomaly between the mean raster and the reference period mean.

Args:
    mean_raster (rast): The mean raster to compare.
    mean_1981_2010 (rast): The reference period mean raster.

Returns:
    list: A list containing the anomaly raster, breaks, and color function for visualization.
"""
compute_anomaly <- function(mean_raster, mean_1981_2010) {
  mean_raster <- filter_raster_values(mean_raster)
  mean_1981_2010 <- filter_raster_values(mean_1981_2010)
  
  anomaly <- mean_raster - mean_1981_2010
  
  minmax_value <- minmax(anomaly)
  min_anomaly <- minmax_value[1]
  max_anomaly <- minmax_value[2]
  
  breaks_neg <- seq(min_anomaly/2-150, -1, length.out = 11)  
  breaks_pos <- seq(1, max_anomaly/2+150, length.out = 11) 
  breaks <- c(min_anomaly,breaks_neg, breaks_pos,max_anomaly)
  
  col_fun <- c("#053061", "#114781", "#1D5FA2", "#2B73B3", "#3A87BD", "#4F9BC7", "#71B0D3", 
               "#93C6DE", "#B1D5E7", "#CCE2EF", "#DEEBF2", "white", "#F8F1ED", "#FBE5D8", "#FCD7C2", 
               "#F8BFA4", "#F4A683", "#E8896C", "#DB6B55", "#CC4C44", "#BD2D35", "#A81529", 
               "#870A24")
  
  return(list(anomaly = anomaly, breaks = breaks, col_fun = col_fun))
}


"""
Save the anomaly plot with overlays.

Args:
    anomaly_data (list): A list containing the anomaly raster, breaks, and color function.
    mean_raster (rast): The mean raster for overlay.
    year (int)
    month (str)

"""
save_anomaly_plot <- function(anomaly_data,mean_raster, year, month) {
  plot(anomaly_data$anomaly, 
       col = anomaly_data$col_fun, 
       breaks = anomaly_data$breaks, legend = FALSE, axes = FALSE, box = TRUE, mar = c(0, 0, 0, 0))
  title(main = sprintf("%s_%s", month,year),line=-5,cex.main = 2)
  plot(mean_raster, col = c(rgb(0, 0, 0, maxColorValue = 255),rgb(119, 119, 119, maxColorValue = 255)), breaks = c(2529,2531,2540),add=TRUE, legend = FALSE, axes = FALSE, box = FALSE)
}

# 4 MAIN ----

"""
Main function to process and visualize the sea ice concentration and anomalies.
"""
json_path <- "galaxy_inputs/galaxy_inputs.json"
json_data <- fromJSON(json_path)

month <- json_data$month
year <- json_data$year
chose <- json_data$ConcentrationXandXorXAnomalies

month_list=resultat <- unlist(strsplit(month, "[,]"))
year_list=resultat <- unlist(strsplit(year, "[,]"))
chose_list=resultat <- unlist(strsplit(chose, "[_]"))

png("outputs/sea_ice.png",width = 790*length(chose_list),height = 830*length(month_list))
par(mfrow=c(length(month_list),length(chose_list)))

for(i in 1:length(month_list)){
    print(i)
    paths <- create_directories(year_list[i], month_list[i])
    month_index <- match(month_list[i], names(months_days))
    
    download_tiff_files(paths$tif_path, year_list[i] ,month_list[i], month_index)
    mean_raster <- compute_monthly_mean(paths$tif_path, year_list[i], month_index)
    
    download_shapefiles(paths$zip_path, paths$shp_path, month_index)
    polygons <- process_shapefiles(paths$shp_path)
    
    color_breaks <- create_colors_and_breaks()
    
    for(txt in chose_list){
        if(txt == "Concentration"){
            save_plot(mean_raster, color_breaks$colors, color_breaks$breaks, polygons, year_list[i], month_list[i])
        }
        if(txt == "Anomalies"){
            download_tiff_1981_2010(paths$tif_mean_path,1981,2010, month_index)
            mean_1981_2010 <- compute_mean_raster_1981_2010(paths$tif_mean_path,month_index)

            anomaly_data=compute_anomaly(mean_raster, mean_1981_2010)
            save_anomaly_plot(anomaly_data,mean_raster, year_list[i], month_list[i])
        }
    }
}
dev.off()
